# Combining Our Local Data Sources

## Combining `.tif` File Data

In [ ]:
from rasterio.plot import plotting_extent
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import rasterio

tif_files = {}
datasets = {}
for i in range (1, 2):
    if i < 10:
        tif_file_wst = f"../Data/gsa/westhem-monthly/PVOUT_0{i}.tif"
    else:
        tif_file_wst = f"../Data/gsa/westhem-monthly/PVOUT_{i}.tif"
    tif_files[i] = tif_file_wst
    
    # Reading the .tif file
    with rasterio.open(tif_file_wst) as ds:
        pvo_data = ds.read(1)  # Reading the first band
        
        # Data dimensions
        h = ds.height
        w = ds.width
        t = ds.transform

        # Data values
        lats = []
        longs = []
        pvo_vals = []

        # Extracting the information of each data points from processed .tif file
        for row in range(h):
            for col in range(w):
                pvo_val = pvo_data[row, col]
                if pvo_val != ds.nodata and not np.isnan(pvo_val):
                    
                    # Converting pixel coords to geographic coords
                    lat, long = ds.xy(t, row, col)
                    lats.append(lat)
                    longs.append(long)
                    pvo_vals.append(pvo_val)

        # Creating a DataFrame
        df = pd.DataFrame({
            'latitude': lats,
            'longitude': longs,
            'pvo': pvo_vals
        })
        datasets[i] = df            

print(datasets[1].head())

Empty DataFrame
Columns: [latitude, longitude, pvo]
Index: []
